#### 1. Install Dependencies

In [1]:
pip install -r ../requirements.txt -q


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show pinecone-client

Name: pinecone-client
Version: 2.2.4
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io/
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Proprietary License
Location: /Users/vdesai/Dev/git/vaibhavdesai137/langchain-pinecone-openai/.venv/lib/python3.11/site-packages
Requires: dnspython, loguru, numpy, python-dateutil, pyyaml, requests, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


#### 2. Verify Pinecone

In [3]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

pinecone.init(
  api_key=os.environ.get("PINECONE_API_KEY"),
  environment=os.environ.get("PINECONE_ENV")
)
pinecone.info.version()

/Users/vdesai/Dev/git/vaibhavdesai137/langchain-pinecone-openai/.venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


VersionResponse(server='2.0.11', client='2.2.4')

#### 3. Pinecone Indexes

In [4]:
idx_name = 'langchain-pinecone'

####################
### create index ###
####################

# only one index allowed in free account
if idx_name not in pinecone.list_indexes():
  pinecone.create_index(name=idx_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
else:
  print(f'Index {idx_name} already exists. Not creating one.')
  
######################
### describe index ###
######################

pinecone.describe_index(idx_name)
pinecone.Index(idx_name).describe_index_stats()

Index langchain-pinecone already exists. Not creating one.


{'dimension': 1536,
 'index_fullness': 5e-05,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

#### 6. Pinecone Vectors

In [6]:
import random
idx_name = 'langchain-pinecone'

######################
### create vectors ###
######################

# create 10 random vectors
# each vector is a list of 1536 floats
vectors = [[random.random() for _ in range(1536)] for _ in range(5)]

# unqiue id for each vector from above
ids = list('ABCDE')
idx = pinecone.Index(idx_name)
idx.upsert(vectors=zip(ids, vectors))

######################
### update vectors ###
######################

# update vectors with id 'A'
updated_vectors = [random.random() for _ in range(1536)]
idx = pinecone.Index(idx_name)
idx.upsert(vectors=[('A', updated_vectors)])

#####################
### fetch vectors ###
#####################

# fetch vectors with id 'A' and 'C'
idx = pinecone.Index(idx_name)
idx.fetch(ids=['A', 'C'])

#####################
### query vectors ###
#####################

# create two random vectors to search for
random_vector = [[random.random() for _ in range(1536)] for _ in range(1)]
idx = pinecone.Index(idx_name)
idx.query(
  vector=random_vector, 
  top_k=2,
  include_values=False
)

{'matches': [{'id': 'A', 'score': 0.756775916, 'values': []},
             {'id': 'D', 'score': 0.748367906, 'values': []}],
 'namespace': ''}

#### 8. Delete Vectors & Indexes

In [ ]:
############################
### delete vectors by id ###
############################

# delete vectors with id 'B' and 'D'
# idx = pinecone.Index(idx_name)
# idx.delete(ids=['B', 'D'])

##########################
### delete all vectors ###
##########################

# idx = pinecone.Index(idx_name)  
# idx.delete_all()

####################
### delete index ###
####################

# if idx_name in pinecone.list_indexes():
#   pinecone.delete_index(idx_name)
# else:
#   print(f'Index {idx_name} does not exist.')

{'matches': [{'id': 'D', 'score': 0.770377219, 'values': []},
             {'id': 'A', 'score': 0.75805676, 'values': []},
             {'id': 'E', 'score': 0.757711649, 'values': []}],
 'namespace': ''}